In [1]:
import os
import sys
import pathlib
import json
import itertools
import codecs
import requests
import copy

In [8]:
def json_make():
    
    stationcode=22715
    date=20201102

    url="https://api.apigw.smt.docomo.ne.jp/ekispertCorp/v1/stationTimetable?APIKEY=45714e327a445136424e74397346526346544a4d6865507a366d7a73342f334c636942663646573162322e&date="+str()data+"&stationCode="+str(stationcode)
    r = requests.get(url)
    data = json.loads(r.text)
    
    resultset={}
    linelist = data['ResultSet']['TimeTable']

    resultset["Station"]=linelist[0]["Station"]["Name"]
    resultset["Linelist"]=[]
    
    result={}
    for line in linelist:
        url="https://api.apigw.smt.docomo.ne.jp/ekispertCorp/v1/stationTimetable?APIKEY=45714e327a445136424e74397346526346544a4d6865507a366d7a73342f334c636942663646573162322e&stationCode="+str(stationcode)+"&date="+str()data+"&code="+str(line['code'])
        r = requests.get(url)
        linedata = json.loads(r.text)
        hour = linedata['ResultSet']['TimeTable']['HourTable']
        result["Line"]=linedata['ResultSet']['TimeTable']['Line']['Name']+linedata['ResultSet']['TimeTable']['Line']['Direction']+"方面"
        result["Linecode"]=linedata['ResultSet']['TimeTable']['code']
        result["HourTable"]=[]
        gokei=0

        if "MinuteTable" in hour:
            result["HourTable"].append({"Hour":hour['Hour'],"number":1})
            gokei=1
        else:
            for h in hour:
                if "MinuteTable" in h:
                    number=len(h['MinuteTable'])
                    if "Minute" in h['MinuteTable']:number-=1
                    result["HourTable"].append({"Hour":h['Hour'],"number":str(number)})
                    gokei+=number
        result["Sum"]=str(gokei)
        
        resultset["Linelist"].append(copy.deepcopy(result))

        # ensure_ascii=Falseをつけることで、日本語エンコードを回避
    #print(resultset)
    expfile = codecs.open("./INresults/in01.json", 'w', 'utf-8')
    json.dump(resultset, expfile, indent=4, ensure_ascii=False)


# 出力フォルダがなければ作成
exp = pathlib.Path('./INresults')
if exp.exists() is False:
    exp.mkdir()

json_make()